In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn.tree 
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import cross_validate
import yfinance as yf

In [5]:
# Download S&P 500 data and 3m,5y,10y,30Y bond yields
sp_500 = yf.download(tickers="^GSPC",start="1995-01-01",interval="1d")
treas_ylds = yf.download(tickers=["^IRX","^FVX","^TNX","^TYX"],start="1995-01-01",interval="1d")

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  4 of 4 completed
